<a href="https://colab.research.google.com/github/wlswjdals/project_braille01/blob/main/Project_braille_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase-admin
!pip install pdfplumber pytesseract Pillow python-pptx python-docx

# Tesseract와 한국어 언어 팩 설치
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!apt-get install -y tesseract-ocr-kor

!pip install firebase-admin
!pip install pdfplumber pytesseract Pillow python-pptx python-docx

In [ ]:

import os
import time
import requests
import pdfplumber
import hashlib
import pytesseract
from PIL import Image  # 이미지 처리
from pptx import Presentation  # PPTX 처리
from docx import Document  # DOCX 처리
import firebase_admin
from firebase_admin import credentials, db
from google.colab import drive

# 구글 드라이브에 마운트
drive.mount('/content/drive')

# Firebase 초기화
cred = credentials.Certificate("/braille-database-firebase-adminsdk-fcowu-411df8ca35.json")
firebase_admin.initialize_app(cred, {
    'storageBucket': 'gs://braille-database.appspot.com',
    'databaseURL': 'https://braille-database-default-rtdb.firebaseio.com/'
})

# 마지막으로 처리한 파일 해시값 저장 (중복 방지)
last_processed_hash = None

# PDF에서 텍스트 추출
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        print(f"OCR 오류 (PDF): {e}")
        return None

# 이미지에서 텍스트 추출
def extract_text_from_image(file_path):
    try:
        img = Image.open(file_path)
        text = pytesseract.image_to_string(img, lang='kor+eng')  # 한국어+영어 지원
        return text
    except Exception as e:
        print(f"OCR 오류 (이미지): {e}")
        return None

# PPTX에서 텍스트 추출
def extract_text_from_pptx(file_path):
    try:
        presentation = Presentation(file_path)
        text = ""
        for slide in presentation.slides:
            for shape in slide.shapes:
                if shape.has_text_frame:
                    text += shape.text + "\n"
        return text
    except Exception as e:
        print(f"OCR 오류 (PPTX): {e}")
        return None

# DOCX에서 텍스트 추출
def extract_text_from_docx(file_path):
    try:
        document = Document(file_path)
        text = ""
        for paragraph in document.paragraphs:
            text += paragraph.text + "\n"
        return text
    except Exception as e:
        print(f"OCR 오류 (DOCX): {e}")
        return None

# 파일 해시값 계산
def calculate_file_hash(file_path):
    """파일 내용의 SHA256 해시값 계산"""
    hash_sha256 = hashlib.sha256()
    try:
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_sha256.update(chunk)
        return hash_sha256.hexdigest()
    except Exception as e:
        print(f"해시 계산 오류: {e}")
        return None

# Firebase에서 다운로드 URL 가져오기 및 파일 다운로드
def get_download_urls():
    ref = db.reference('files')
    files = ref.get()

    if files:
        # 가장 최신 파일 데이터 가져오기
        last_file = list(files.items())[-1]
        file_data = last_file[1]
        file_name = file_data.get('fileName')
        file_url = file_data.get('fileUrl')
        return file_name, file_url
    else:
        print("Firebase 데이터베이스에 파일이 없습니다.")
        return None, None

# 파일 다운로드
def download_file(file_url, destination_path):
    try:
        response = requests.get(file_url)
        if response.status_code == 200:
            with open(destination_path, 'wb') as file:
                file.write(response.content)
            print(f"파일 다운로드 완료: {destination_path}")
            return True
        else:
            print(f"파일 다운로드 실패: HTTP {response.status_code}")
            return False
    except Exception as e:
        print(f"파일 다운로드 중 오류 발생: {e}")
        return False

# 파일 유형에 따라 OCR 처리
def process_file(file_name, file_path):
    print(f"OCR 처리 시작: {file_name}")
    text = None

    if file_name.lower().endswith('.pdf'):
        text = extract_text_from_pdf(file_path)
    elif file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        text = extract_text_from_image(file_path)
    elif file_name.lower().endswith('.pptx'):
        text = extract_text_from_pptx(file_path)
    elif file_name.lower().endswith('.docx'):
        text = extract_text_from_docx(file_path)
    else:
        print(f"지원하지 않는 파일 형식: {file_name}")
        return

    if text:
        # 30글자 제한
        limited_text = text[:30]
        print(f"OCR 결과 (30글자 제한): {limited_text}")
        # Firebase에 결과 저장
        ref = db.reference('message')
        ref.set(limited_text)
        print("Firebase에 OCR 결과 저장 완료.")
    else:
        print(f"OCR 실패: {file_name}")

# Firebase에서 최신 파일 가져오기 및 OCR 실행
def monitor_and_process():
    global last_processed_hash
    print("Firebase 파일 감시 시작...")
    while True:
        file_name, file_url = get_download_urls()
        if file_name and file_url:
            # 파일 경로 설정
            destination_path = f"/content/{file_name}"
            if download_file(file_url, destination_path):  # 파일 다운로드 성공 시
                # 파일 해시값 계산
                current_hash = calculate_file_hash(destination_path)
                if current_hash != last_processed_hash:
                    print(f"새 파일 또는 파일 내용 변경 감지: {file_name}")
                    process_file(file_name, destination_path)  # OCR 처리
                    last_processed_hash = current_hash  # 처리 완료 파일의 해시값 갱신
                else:
                    print("파일 내용이 이전과 동일합니다. OCR을 건너뜁니다.")
        else:
            print("새로운 파일이 없습니다.")
        time.sleep(10)  # 10초 간격으로 Firebase 확인

# OCR 감시 및 실행 시작
monitor_and_process()
